### ***Load and preprocess data***

In [151]:
import pandas as pd
import numpy as np

In [152]:
# Load datasets
books = pd.read_csv('../data/BX-Books.csv', sep=';', on_bad_lines='skip', encoding='latin-1')

C:\Users\emon1\AppData\Local\Temp\ipykernel_5220\3948767560.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('../data/BX-Books.csv', sep=';', on_bad_lines='skip', encoding='latin-1')


In [153]:
books.head(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...


In [154]:
books.shape

(271360, 8)

In [155]:
users = pd.read_csv('../data/BX-Users.csv', sep=';', on_bad_lines='skip', encoding='latin-1')

In [156]:
users.head(2)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0


In [157]:
users.shape

(278858, 3)

In [158]:
ratings = pd.read_csv('../data/BX-Book-Ratings.csv', sep=';', on_bad_lines='skip', encoding='latin-1')

In [159]:
ratings.head(2)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5


In [160]:
ratings.shape

(1149780, 3)

### ***Rename books data***

In [161]:
books = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-L']]

In [162]:
books.rename(columns={
        'Book-Title': 'title',
        'Book-Author': 'author',
        'Year-Of-Publication': 'year',
        'Publisher': 'publisher',
        'Image-URL-L': 'img_url'
    }, inplace=True)

### ***Rename users data***

In [163]:
users.rename(columns={
        'User-ID': 'user_id',
        'Location': 'location',
        'Age': 'age'
    }, inplace=True)

### ***Rename ratings data***

In [164]:
ratings.rename(columns={
        'User-ID': 'user_id',
        'Book-Rating': 'rating'
    }, inplace=True)

### ***Filter active users (users with more than 200 ratings)***

In [165]:
user_ratings_count = ratings['user_id'].value_counts()

In [166]:
user_ratings_count

user_id
11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: count, Length: 105283, dtype: int64

### ***Filter active users (users with more than 200 ratings)***

In [167]:
user_ratings_count = ratings['user_id'].value_counts()

In [168]:
user_ratings_count

user_id
11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: count, Length: 105283, dtype: int64

In [169]:
active_users = user_ratings_count[user_ratings_count > 200].index

In [170]:
active_users

Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352, 110973,
       235105,
       ...
       260183,  73681,  44296, 155916,   9856, 274808,  28634,  59727, 268622,
       188951],
      dtype='int64', name='user_id', length=899)

In [171]:
active_users = user_ratings_count[user_ratings_count > 200].index

In [172]:
active_users

Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352, 110973,
       235105,
       ...
       260183,  73681,  44296, 155916,   9856, 274808,  28634,  59727, 268622,
       188951],
      dtype='int64', name='user_id', length=899)

In [173]:
ratings = ratings[ratings['user_id'].isin(active_users)]

In [174]:
ratings

,user_id,ISBN,rating
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8
1459,277427,0030615321,0
1460,277427,0060002050,0
...,...,...,...
1147612,275970,3829021860,0
1147613,275970,4770019572,0
1147614,275970,896086097,0
1147615,275970,9626340762,8


### ***Merge ratings with books***

In [175]:
ratings_with_books = ratings.merge(books, on="ISBN")

In [176]:
ratings_with_books.head(2)

,user_id,ISBN,rating,title,author,year,publisher,img_url
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
1,277427,0026217457,0,Vegetarian Times Complete Cookbook,Lucy Moll,1995,John Wiley &amp; Sons,http://images.amazon.com/images/P/0026217457.0...


In [177]:
ratings_with_books.shape

(487671, 8)

### ***Filter popular books (books with at least 50 ratings)***

In [178]:
book_ratings_count = ratings_with_books.groupby('title')['rating'].count().reset_index()

In [179]:
book_ratings_count.rename(columns={'rating': 'num_ratings'}, inplace=True)

In [180]:
book_ratings_count.head(2)

,title,num_ratings
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1


In [181]:
final_rating = ratings_with_books.merge(book_ratings_count, on='title')

In [182]:
final_rating.head(2)

,user_id,ISBN,rating,title,author,year,publisher,img_url,num_ratings
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,277427,0026217457,0,Vegetarian Times Complete Cookbook,Lucy Moll,1995,John Wiley &amp; Sons,http://images.amazon.com/images/P/0026217457.0...,7


In [183]:
final_rating = final_rating[final_rating['num_ratings'] >= 50]

In [184]:
final_rating.head(2)

,user_id,ISBN,rating,title,author,year,publisher,img_url,num_ratings
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
13,277427,0060930535,0,The Poisonwood Bible: A Novel,Barbara Kingsolver,1999,Perennial,http://images.amazon.com/images/P/0060930535.0...,133


In [185]:
final_rating.shape

(61853, 9)

In [186]:
final_rating.drop_duplicates(['user_id', 'title'], inplace=True)

In [187]:
final_rating.shape

(59850, 9)

### ***Prepare content data***

In [188]:
books_content = books.drop_duplicates('title')

In [189]:
books_content.shape

(242135, 6)

In [190]:
books_content = books_content[books_content['title'].isin(final_rating['title'])]
books_content['content_features'] = (
    books_content['title'] + ' ' + 
    books_content['author'] + ' ' + 
    books_content['publisher'].fillna('') + ' ' +
    books_content['year'].astype(str)
)

In [191]:
books_content.head(2)

,ISBN,title,author,year,publisher,img_url,content_features
5,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,The Kitchen God's Wife Amy Tan Putnam Pub Grou...
18,0440234743,The Testament,John Grisham,1999,Dell,http://images.amazon.com/images/P/0440234743.0...,The Testament John Grisham Dell 1999


In [192]:
books_content.shape

(742, 7)

### ***Build Collaborative Filtering Model***

In [193]:
from scipy.sparse import csr_matrix

In [194]:
# Create user-item matrix
book_pivot = final_rating.pivot_table(index='title', columns='user_id', values='rating').fillna(0)
book_sparse = csr_matrix(book_pivot.values)

In [195]:
from sklearn.neighbors import NearestNeighbors

In [196]:
# Build KNN model
cf_model = NearestNeighbors(metric='cosine', algorithm='brute')
cf_model.fit(book_sparse)

NearestNeighbors(algorithm='brute', metric='cosine')

### ***Build Content-Based Model***

In [197]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [198]:
# TF-IDF Vectorizer
tfidf = TfidfVectorizer(stop_words='english', max_features=10000)

In [199]:
tfidf_matrix = tfidf.fit_transform(books_content['content_features'])

In [200]:
from sklearn.metrics.pairwise import cosine_similarity

In [201]:
content_sim_matrix = cosine_similarity(tfidf_matrix)

In [202]:
# Create title to index mapping
title_to_idx = pd.Series(books_content.index, index=books_content['title'])

In [203]:
title_to_idx.drop_duplicates(inplace=True)

### ***Recommendation Functions***

### ***Generate collaborative filtering recommendations***

In [204]:
def collaborative_recommendations(book_title, top_n=10):
    """Generate collaborative filtering recommendations"""
    try:
        if book_title not in book_pivot.index:
            return []
            
        book_idx = np.where(book_pivot.index == book_title)[0][0]
        distances, indices = cf_model.kneighbors(
            book_pivot.iloc[book_idx, :].values.reshape(1, -1),
            n_neighbors=top_n+1)
        
        recs = []
        for i in range(1, len(indices.flatten())):
            title = book_pivot.index[indices.flatten()[i]]
            book_info = books_content[books_content['title'] == title]
            if book_info.empty:
                book_info = books[books['title'] == title]
                if book_info.empty:
                    continue
            
            book_info = book_info.iloc[0]
            
            img_url = book_info['img_url']
            if not isinstance(img_url, str) or not img_url.startswith('http'):
                img_url = "https://via.placeholder.com/150x220?text=No+Image"
            
            recs.append({
                'title': title,
                'author': book_info['author'],
                'year': book_info['year'],
                'publisher': book_info['publisher'],
                'image_url': img_url,
                'score': (1 - distances.flatten()[i]),
                'type': 'collaborative'
            })
        
        return recs[:top_n]
    
    except Exception as e:
        print(f"Error in collaborative recommendations: {e}")
        return []

### ***Generate content-based recommendations***

In [205]:
def content_recommendations(book_title, top_n=10):
    """Generate content-based recommendations"""
    try:
        if book_title not in title_to_idx:
            return []
            
        cb_idx = title_to_idx[book_title]
        sim_scores = list(enumerate(content_sim_matrix[cb_idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:top_n+1]
        
        recs = []
        for i, score in sim_scores:
            title = books_content['title'].iloc[i]
            book_info = books_content[books_content['title'] == title].iloc[0]
            
            img_url = book_info['img_url']
            if not isinstance(img_url, str) or not img_url.startswith('http'):
                img_url = "https://via.placeholder.com/150x220?text=No+Image"
            
            recs.append({
                'title': title,
                'author': book_info['author'],
                'year': book_info['year'],
                'publisher': book_info['publisher'],
                'image_url': img_url,
                'score': score,
                'type': 'content'
            })
        
        return recs[:top_n]
    
    except Exception as e:
        print(f"Error in content recommendations: {e}")
        return []

### ***Generate hybrid recommendations***

In [206]:
def hybrid_recommendations(book_title, cf_weight=0.6, cb_weight=0.4, top_n=10):
    """Generate hybrid recommendations"""
    try:
        cf_recs = collaborative_recommendations(book_title, top_n*2)
        cb_recs = content_recommendations(book_title, top_n*2)
        
        if not cf_recs and not cb_recs:
            return []
        
        combined_scores = {}
        
        for rec in cf_recs:
            combined_scores[rec['title']] = {
                'data': rec,
                'score': rec['score'] * cf_weight
            }
        
        for rec in cb_recs:
            if rec['title'] in combined_scores:
                combined_scores[rec['title']]['score'] += rec['score'] * cb_weight
            else:
                combined_scores[rec['title']] = {
                    'data': rec,
                    'score': rec['score'] * cb_weight
                }
        
        sorted_recs = sorted(combined_scores.values(), key=lambda x: x['score'], reverse=True)
        
        final_recs = []
        for rec in sorted_recs[:top_n]:
            final_rec = rec['data'].copy()
            final_rec['score'] = rec['score']
            final_rec['type'] = 'hybrid'
            final_recs.append(final_rec)
        
        return final_recs
    
    except Exception as e:
        print(f"Error in hybrid recommendations: {e}")
        return []

### ***Save Models and Data***

In [207]:
import pickle

In [208]:
def save_models():
    """Save all models and data for Flask app"""
    models_dir = '../models'
    import os
    os.makedirs(models_dir, exist_ok=True)
    
    pickle.dump(cf_model, open(f'{models_dir}/cf_model.pkl', 'wb'))
    pickle.dump(book_pivot, open(f'{models_dir}/book_pivot.pkl', 'wb'))
    pickle.dump(tfidf, open(f'{models_dir}/tfidf_vectorizer.pkl', 'wb'))
    pickle.dump(content_sim_matrix, open(f'{models_dir}/content_sim_matrix.pkl', 'wb'))
    pickle.dump(title_to_idx, open(f'{models_dir}/title_to_idx.pkl', 'wb'))
    pickle.dump(books_content, open(f'{models_dir}/books_content.pkl', 'wb'))
    pickle.dump(final_rating, open(f'{models_dir}/final_rating.pkl', 'wb'))
    pickle.dump(books, open(f'{models_dir}/books_data.pkl', 'wb'))
    
    print("All models and data saved successfully!")

save_models()

All models and data saved successfully!


### ***Test the System***

In [209]:
if __name__ == "__main__":
    test_books = ["The Kiss"]
    
    for book in test_books:
        print("\n" + "="*80)
        print(f"Testing recommendations for: {book}")
        print("="*80)
        
        print("\nHybrid Recommendations:")
        hybrid_recs = hybrid_recommendations(book)
        for i, rec in enumerate(hybrid_recs[:2], 1):
            print(f"{i}. {rec['title']} by {rec['author']} (Score: {rec['score']:.2f})")
            print(f"   Image URL: {rec['image_url']}\n")
        
        print("\nCollaborative Filtering Recommendations:")
        cf_recs = collaborative_recommendations(book)
        for i, rec in enumerate(cf_recs[:2], 1):
            print(f"{i}. {rec['title']} by {rec['author']} (Score: {rec['score']:.2f})")
            print(f"   Image URL: {rec['image_url']}\n")
        
        print("\nContent-Based Recommendations:")
        cb_recs = content_recommendations(book)
        for i, rec in enumerate(cb_recs[:2], 1):
            print(f"{i}. {rec['title']} by {rec['author']} (Score: {rec['score']:.2f})")
            print(f"   Image URL: {rec['image_url']}\n")


Testing recommendations for: The Kiss

Hybrid Recommendations:
1. Riptide by Catherine Coulter (Score: 0.25)
   Image URL: http://images.amazon.com/images/P/0515130966.01.LZZZZZZZ.jpg

2. The Reef by Nora Roberts (Score: 0.24)
   Image URL: http://images.amazon.com/images/P/051512608X.01.LZZZZZZZ.jpg


Collaborative Filtering Recommendations:
1. The Ghost by Danielle Steel (Score: 0.29)
   Image URL: http://images.amazon.com/images/P/0440224853.01.LZZZZZZZ.jpg

2. The Reef by Nora Roberts (Score: 0.28)
   Image URL: http://images.amazon.com/images/P/051512608X.01.LZZZZZZZ.jpg


Content-Based Recommendations:
1. Riptide by Catherine Coulter (Score: 0.64)
   Image URL: http://images.amazon.com/images/P/0515130966.01.LZZZZZZZ.jpg

2. Hemlock Bay by Catherine Coulter (Score: 0.59)
   Image URL: http://images.amazon.com/images/P/0515133302.01.LZZZZZZZ.jpg

